In [1]:
import pandas as pd
import numpy as np
import haversine as hs

pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
pd.set_option('display.precision', 0)
pd.options.display.float_format = "{:,.0f}".format

In [2]:
data = pd.read_csv("./data/kc_house_data.csv")
df_fortune_500s = pd.read_excel("./data/Fortune500LatsLongs.xlsx")

In [3]:
df_fortune_500s['tup_lat_Long_500'] = list(zip(df_fortune_500s.Lat, df_fortune_500s.Long, df_fortune_500s.Company))

data['tup_lat_Long'] = list(zip(data.lat, data.long, data.id))

In [4]:
df_Dist_From_Fortune500 = pd.DataFrame(columns = ['id', 'Company', 'Distance'])

distanceConsideredClose = 7
listofValues = []

#looping Through Main mdf
for HLat, HLong, HouseParcel in data['tup_lat_Long']:
    #print(HouseParcel)
    
    #looping through df that contains interesting places 
    for fiveH_Lat, fiveH_Long, fiveH_Company in df_fortune_500s['tup_lat_Long_500']:
        
        #getting the distance between current house and point of interest
        distance = round(hs.haversine((HLat,HLong),(fiveH_Lat,fiveH_Long), unit='mi'),2)
    
        #if point distance < 6 miles insert into a dataframe 
#         if distance < distanceConsideredClose:
        values = [HouseParcel, fiveH_Company, distance]
        listofValues.append(values)

keys = ['id', 'Company', 'Distance']
df_Dist_From_500 = pd.DataFrame(listofValues,columns = keys)


In [5]:
df_Dist_From_500["FortuneFiveBin"] = df_Dist_From_500['Distance'].apply(lambda x : "Under10" if x <10 else "Over10" if x <20 else "Over20")
df_Dist_From_500["FortuneFiveBin_2"] = df_Dist_From_500['Distance'].apply(lambda x : "Under7" if x <=7 else "Over7")

df_Dist_From_500.fillna(0, inplace = True)

In [6]:
df_Counts_of_Fortune500_FF_3Buckets = df_Dist_From_500[["id", "FortuneFiveBin"]].groupby(["id", "FortuneFiveBin"]).size().reset_index(name='counts')
df_Counts_of_Fortune500_FF_2Buckets = df_Dist_From_500[["id", "FortuneFiveBin_2"]].groupby(["id", "FortuneFiveBin_2"]).size().reset_index(name='counts')


In [7]:
df_Counts_of_Fortune500_FF_3Buckets_pivot = pd.pivot_table(df_Counts_of_Fortune500_FF_3Buckets, index='id', columns=['FortuneFiveBin'], values='counts', aggfunc='max')
df_Counts_of_Fortune500_FF_2Buckets_pivot = pd.pivot_table(df_Counts_of_Fortune500_FF_2Buckets, index='id', columns=['FortuneFiveBin_2'], values='counts', aggfunc='max')

In [8]:
df_Counts_of_Fortune500_FF_3Buckets_pivot.reset_index()
df_Counts_of_Fortune500_FF_2Buckets_pivot.reset_index()
df_Counts_of_Fortune500_pivot = df_Counts_of_Fortune500_FF_3Buckets_pivot.merge(df_Counts_of_Fortune500_FF_2Buckets_pivot, on="id")

In [9]:
df_Counts_of_Fortune500_pivot = df_Counts_of_Fortune500_pivot.fillna(0)

# Export Files To Excel

In [ ]:
df_Counts_of_Fortune500_pivot.to_excel("df_Counts_of_Fortune500_CrossTab.xlsx")